## Example showing the use of Xarray to read OpenDAP Netcdf files from Sentinel

In [ ]:
%matplotlib widget

import numpy as np
import xarray as xa
import matplotlib.pyplot as plt

We have now imported some libraries and will load some Sentinel 1 OpenDAP NetCDF
data from https://satellittdata.no

In [ ]:
# Old sentinel 2 data
# url = 'https://nbstds.met.no/thredds/dodsC/DC-S2-32VMK-2020_BB'
# Fra satellittdata.no


url = 'https://nbstds.met.no/thredds/dodsC/NBS/S1C/2025/10/16/IW/S1C_IW_GRDH_1SDV_20251016T054906_20251016T054931_004583_009104_9116.nc'

dset = xa.open_dataset(url+'#fillmismatch') # opening OpenDAP NetCDF dataset


Having opened the dataset we can how have a look at the header

In [ ]:
display(dset)

In [ ]:
display(dset.Amplitude_VV)

**What is Amplitude_VV**

We can plot part of Amplitude_VV to see it better

In [ ]:
# East - west
xmin = 9000
xmax = 11000

# North - south
ymin = 0
ymax = 2000

plt.figure()
view =dset.Amplitude_VV[0,ymin:ymax,xmin:xmax]
view.plot(cmap='Greys',vmax=500)
plt.show()



In [ ]:

extent = np.asarray(
            [dset.lon[ymin,xmin].data, dset.lon[ymin,xmax].data,
            dset.lat[ymax,xmax].data, dset.lat[ymin,xmin].data]
         )
